<a href="https://colab.research.google.com/github/susantaghosh1/nlp-notebooks/blob/main/custom_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.6 MB/s eta 0:00:00


In [28]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.234
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass

In [14]:
def get_cibil_score(input):
    print(f'\nreceived input in function {input.lower()}')
    cb ={'sus':10,'shiju': 20,'ricky' : 30,'lory' : 40}
    return {'name': input.lower(),'cibil_score' : cb[input.lower()]}

def net_asset(input):
    print(f'\nreceived input in net asset function {input.lower()}')
    cb ={'sus':'42M USD','shiju': '50M USD','ricky' : '53M USD','lory' : '60M USD'}
    return {'name': input.lower(),'asset' : cb[input.lower()]}

tools = [
    Tool(
        name = "Get Cibil Score",
        func=get_cibil_score,
        description="useful for when you need to answer cibil score related questions. Always provide name as  input"
    ),
     Tool(
        name = "Get net asset",
        func=net_asset,
        description="useful for when you need to answer net asset related questions. Always provide name as  input"
    )
]

In [15]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate financial professional. You have access to the following tools:

{tools}

You don't know anything about cibil score and net asset related things and should use a tool for questions about these topics.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin! Remember to answer as a compansionate financial professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [16]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [17]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [18]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [19]:
output_parser = CustomOutputParser()

In [20]:
OPENAI_API_KEY = getpass()
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

··········


In [21]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [132]:
agent_executor

AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, agent=LLMSingleActionAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=CustomPromptTemplate(input_variables=['input', 'intermediate_steps'], output_parser=None, partial_variables={}, template="Answer the following questions as best you can, but speaking as compasionate financial professional. You have access to the following tools:\n\n{tools}\n\nYou don't know anything about cibil score and net asset related things and should use a tool for questions about these topics.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThoug

In [133]:
agent_executor.agent.llm_chain.prompt.template

"Answer the following questions as best you can, but speaking as compasionate financial professional. You have access to the following tools:\n\n{tools}\n\nYou don't know anything about cibil score and net asset related things and should use a tool for questions about these topics.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nThese were previous tasks you completed:\n\n\n\nBegin! Remember to answer as a compansionate financial professional when giving your final answer.\n\nQuestion: {input}\n{agent_scratchpad}"

In [22]:
agent_executor.run("What's cibil score of lory, sus and shiju")



> Entering new AgentExecutor chain...
Thought: I need to find out the Cibil score of Lory, Sus, and Shiju.
Action: Get Cibil Score
Action Input: Lory
received input in function lory


Observation:{'name': 'lory', 'cibil_score': 40}
I have obtained the Cibil score for Lory. Now I need to find the Cibil score for Sus and Shiju.
Action: Get Cibil Score
Action Input: Sus
received input in function sus


Observation:{'name': 'sus', 'cibil_score': 10}


I have obtained the Cibil score for Sus. Now I need to find the Cibil score for Shiju.
Action: Get Cibil Score
Action Input: Shiju
received input in function shiju


Observation:{'name': 'shiju', 'cibil_score': 20}


I now know the final answer. 
Final Answer: The Cibil score of Lory is 40, the Cibil score of Sus is 10, and the Cibil score of Shiju is 20.

> Finished chain.


'The Cibil score of Lory is 40, the Cibil score of Sus is 10, and the Cibil score of Shiju is 20.'

In [23]:
agent_executor.run(" cibil score and net asset of shiju, sus and lory and return those in a structured json format")



> Entering new AgentExecutor chain...
Thought: I need to gather the cibil score and net asset information for Shiju, Sus, and Lory.
Action: Get Cibil Score
Action Input: Shiju
received input in function shiju


Observation:{'name': 'shiju', 'cibil_score': 20}


I have obtained the Cibil score for Shiju. Now I need to get the net asset information for Shiju.
Action: Get net asset
Action Input: Shiju
received input in net asset function shiju


Observation:{'name': 'shiju', 'asset': '50M USD'}


I have obtained the net asset information for Shiju. Now I need to repeat the same process for Sus and Lory.
Action: Get Cibil Score
Action Input: Sus
received input in function sus


Observation:{'name': 'sus', 'cibil_score': 10}


I have obtained the Cibil score for Sus. Now I need to get the net asset information for Sus.
Action: Get net asset
Action Input: Sus
received input in net asset function sus


Observation:{'name': 'sus', 'asset': '42M USD'}


I have obtained the net asset information for Sus. Now I need to repeat the same process for Lory.
Action: Get Cibil Score
Action Input: Lory
received input in function lory


Observation:{'name': 'lory', 'cibil_score': 40}


I have obtained the Cibil score for Lory. Now I need to get the net asset information for Lory.
Action: Get net asset
Action Input: Lory
received input in net asset function lory


Observation:{'name': 'lory', 'asset': '60M USD'}


I have obtained the net asset information for Lory. Now I have gathered all the necessary information for Shiju, Sus, and Lory.
Thought: I now know the final answer.
Final Answer: {'Shiju': {'cibil_score': 20, 'net_asset': '50M USD'}, 'Sus': {'cibil_score': 10, 'net_asset': '42M USD'}, 'Lory': {'cibil_score': 40, 'net_asset': '60M USD'}}

> Finished chain.


"{'Shiju': {'cibil_score': 20, 'net_asset': '50M USD'}, 'Sus': {'cibil_score': 10, 'net_asset': '42M USD'}, 'Lory': {'cibil_score': 40, 'net_asset': '60M USD'}}"

In [24]:
agent_executor.run("What's total asset of lory and then tell about sus and then tell about shiju")



> Entering new AgentExecutor chain...


Thought: I need to find out the total asset of Lory first, and then find out about Sus and Shiju individually.
Action: Get net asset
Action Input: Lory
received input in net asset function lory


Observation:{'name': 'lory', 'asset': '60M USD'}


Now I have the total asset of Lory, I can find out about Sus and Shiju individually.
Action: Get net asset
Action Input: Sus
received input in net asset function sus


Observation:{'name': 'sus', 'asset': '42M USD'}


Now I have the net asset of Sus. I can find out about Shiju individually.
Action: Get net asset
Action Input: Shiju
received input in net asset function shiju


Observation:{'name': 'shiju', 'asset': '50M USD'}


I now know the final answer. 

Final Answer: The total asset of Lory is 60M USD. Sus has a net asset of 42M USD and Shiju has a net asset of 50M USD.

> Finished chain.


'The total asset of Lory is 60M USD. Sus has a net asset of 42M USD and Shiju has a net asset of 50M USD.'

In [26]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=edbf1f0635c28edb205851bf55a9a7690b6c5a332df38664f6165744c8e4783c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [33]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

llm = OpenAI(temperature=0)
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [35]:
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff and the U.S. Navy admiral, find the ambassador to the United Kingdom, then find the President they served under.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought: David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom. I need to search U.S. Navy admiral and find the ambassador to the United Kingdom.
Action: Search[U.S. Navy admiral]


Observation: Admiral of the Navy was the highest-possible rank in the United States Navy, prior to the creation of fleet admiral in 1944. The rank is considered to be at least equivalent to that of a five-star admiral, with Admiral George Dewey being the only officer to be appointed to the rank.
Thought: The U.S. Navy admiral who served as the ambassador to the United Kingdom is Admiral William J. Crowe. I need to search William J. Crowe and find the President they served under.
Action: Search[William J. Crowe]


Observation: William James Crowe Jr. (January 2, 1925 – October 18, 2007) was a United States Navy admiral and diplomat who served as the 11th chairman of the Joint Chiefs of Staff under Presidents Ronald Reagan and George H. W. Bush, and as the ambassador to the United Kingdom and Chair of the Intelligence Oversight Board under President Bill Clinton.
Thought:

 William J. Crowe served as the ambassador to the United Kingdom under President Bill Clinton, so the answer is Bill Clinton.
Action: Finish[Bill Clinton]

> Finished chain.


'Bill Clinton'

In [34]:
react.run("What were Archimedes last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other

'Do not disturb my circles'

In [36]:
react.run("Which genus of moth in the world's seventh-largest country contains only one species?")



> Entering new AgentExecutor chain...
Thought: I need to search the world's seventh-largest country, find the genus of moth that contains only one species.
Action: Search[world's seventh-largest country]
Observation: This is a pair of lists of terrestrial lakes with a surface area of more than approximately 3,000 square kilometres (1,200 sq mi), ranked by area, excluding reservoirs and lagoons.
The area of some lakes can vary considerably over time, either seasonally or from year to year. This is especially true of salt lakes in arid climates.
This list therefore excludes seasonal lakes such as Kati Thanda–Lake Eyre (maximum area 9,500 km2, 3,700 sq mi), Mar Chiquita Lake (Córdoba) (maximum area 6,000 km2, 2,300 sq mi), Lake Torrens (maximum area 5,745 km2, 2,218 sq mi) and Great Salt Lake (maximum area, 1988, 8,500 km2, 3,300 sq mi).
The list is divided in two: all lakes as conventionally defined down to 3,000 square kilometres (1,200 sq mi), and the largest lakes under a geological


Observation: This is a list of the world's countries and their dependencies by land, water, and total area, ranked by total area.
Entries in this list include, but are not limited to, those in the ISO 3166-1 standard, which includes sovereign states and dependent territories. All 193 member states of the United Nations plus the two observer states are given a rank number. Largely unrecognised states not in ISO 3166-1 are included in the list in ranked order. The areas of such largely unrecognised states are in most cases also included in the areas of the more widely recognised states that claim the same territory; see the notes in the "notes" column for each country for clarification.
Not included in the list are individual country claims to parts of the continent of Antarctica or entities such as the European Union that have some degree of sovereignty but do not consider themselves to be sovereign countries or dependent territories.
This list includes three measurements of area:
Thou

 The seventh-largest country is India. I need to search India and find the genus of moth that contains only one species.
Action: Search[India, genus of moth]


Observation: Could not find [India, genus of moth]. Similar: ['List of moths of India', 'Attacus atlas', 'Vigna aconitifolia', 'Baorisa hieroglyphica', 'Calyptra (moth)', 'Musa (genus)', 'Bombyx mori', 'Phalaenopsis', 'Asthana', 'Indianmeal moth']
Thought:

 To find the genus of moth, I can search List of moths of India.
Action: Search[List of moths of India]

...

 I can look through the list and find the genus of moth that contains only one species.
Action: Lookup[genus of moth]
Observation: No Results
Thought:

 I need to look through the list and find the genus of moth that contains only one species.
Action: Lookup[list]
Observation: (Result 1/3) The following is a list of the moths of India. It is estimated that approximately 10,000 species of moths exist in India. 
Thought:

 I can look through the list and find the genus of moth that contains only one species.
Action: Lookup[list]
Observation: (Result 2/3) 
== Family lists for Indian moths ==
Alucitidae
Bombycidae
Brachodidae
Brahmaeidae
Choreutidae
Cosmopterigidae
Cossidae
Crambidae
Depressariidae
Drepanidae
Elachistidae
Urodeta jurateae Sruoga & Rociene, 2018
Urodeta pectena Sruoga & Rociene, 2018
Erebidae
Eupterotidae
Gelechiidae
Geometridae
Glyphipterigidae
Gracillariidae
Hepialidae
Immidae
Lasiocampidae
Lecithoceridae
Limacodidae
Lymantriidae
Noctuidae
Nolidae
Notodontidae
Oecophoridae
Pantheidae
Peleopodidae
Phaudidae
Psychidae
Pterophoridae
Pyralidae
Saturniidae
Sesiidae
Sphingidae
Thyrididae
Tineidae
Tortricidae
Uraniidae
Xyloryctidae
Yponomeutidae
Zygaenidae
Thought:

 Urodeta jurateae and Urodeta pectena are the only species in the genus Urodeta, so the answer is Urodeta.
Action: Finish[Urodeta]

> Finished chain.


'Urodeta'